In [44]:
%matplotlib inline
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cvxopt
from cvxopt import solvers
from cvxopt import matrix

#get data
def GetData(filename):
    dataframe = pd.read_csv(filename, header=None)
    dataset = dataframe.values
    Col=len(dataset[0])-1
    X= dataset[:, 0:Col]
    Y=dataset[:, Col]
    return X,Y
#dual problem , fit train data ......optimize Ws
def Fit(x,y):
    #obtain kernel
    row = len(x)
    col = len(x[0])
    K=y[:, None] * x
    K = np.dot(K, K.T)
    P = matrix(K)
    q = matrix(-np.ones((row, 1)))
    G = matrix(-np.eye(row))
    h = matrix(np.zeros(row))
    A = matrix(y.reshape(1, row))
    b = matrix(0.0)
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    return sol,alphas
# caculate accuracy 
def Accuacy(X,y,W,b):
    Train_predict=np.sign(np.dot(X,W)+b)
    '''Train_predict=np.zeros(len(X))
    for j in range (len(X)):
        if np.dot(X[j],Ws)+b>0:
            Train_predict[j]=1
        else :
            if np.dot(X[j],Ws)+b<0:
                Train_predict[j]=-1
    '''
    sum=0.00
    for i in range(len(X)):
        if  Train_predict[i]==y[i] :
            sum+=1
    return  sum/(len(X))

filename='/Users/xing/Desktop/test/source_train.csv'
X,y=GetData(filename)
#fit train data to SVM classifier
sol,a=Fit(X,y)
#get Ws and b
Ws=np.sum(a * y[:, None] * X, axis = 0)
Ws=abs(Ws)
b=0
for x in range (len(a)):
    b+=(y[x]-np.dot(X[x],Ws))
bias=b/len(a)

#caculate test data accuracy by using Ws and b
filename='/Users/xing/Desktop/test/source_test.csv'
X_test,y_test=GetData(filename)
print  "Source weight Ws=",Ws
print "Source testing data accuracy =" ,Accuacy(X_test,y_test,Ws,bias)

#using optimize Ws, do domain adaptataion SVM. The  different is coefficient q,G,h, which need to recaculate
def fit_domainsvm(x,y,C,B):
    row = len(x)
    col = len(x[0])
    K=y[:, None] * x
    K = np.dot(K, K.T)
    P = matrix(K)
    #get new q 
    tmp_B=np.dot((y[:, None] * X),Ws)
    tmp_B=tmp_B.reshape(row,1)
    tmp=np.ones((row, 1))
    q=matrix(-(tmp-B*tmp_B))
    A = matrix(y.reshape(1, row))
    b = matrix(0.0)
    #get new G and h
    tmp1 = np.diag(np.ones(row) * -1)
    tmp2 = np.identity(row)
    G = matrix(np.vstack((tmp1, tmp2)))
    tmp1 = np.zeros(row)
    tmp2 = np.ones(row) *C
    h = matrix(np.hstack((tmp1, tmp2)))
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    return sol,alphas

#get target train data and fit with C=2, B=100
filename='/Users/xing/Desktop/test/target_train.csv'
X,y=GetData(filename)
C=2
B=100
sol,a=fit_domainsvm(X,y,C,B)
#get Weight and Bias
Wt=np.sum(a * y[:, None] * X, axis = 0)
#interception b
b=0
for x in range (len(a1)):
    b+=(y[x]-np.dot(X[x],Wt))
bias=b/len(a1)

#caculate test data accuracy 
filename='/Users/xing/Desktop/test/target_test.csv'
X_test,y_test=GetData(filename)
print 'Target weight Wt=',Wt
print  'Target test data accuracy =',Accuacy(X_test,y_test,Wt,bias)


Source weight Ws= [6.10351562e-05 9.15527344e-05]
Source testing data accuracy = 0.95
Target weight Wt= [1.6648273  0.81585871]
Target test data accuracy = 0.98
